In [2]:
import requests, json
import pandas as pd
import numpy as np
from typing import Dict, List
from datetime import datetime, timedelta

In [101]:
class PanelScraper:
    def __init__(self):
        self.api_prov_id_  : str  = "https://panelharga.badanpangan.go.id/data/provinsi-by-levelharga/3/{}"
        self.api_kab_id_   : str  = "https://panelharga.badanpangan.go.id/data/kabkota-by-levelharga/3/{}/{}"
        self.api_prov_prices_ : str = "https://panelharga.badanpangan.go.id/data/provinsi-range-by-levelharga/{}/3/{}"
        self.api_kab_prices_     : str  = "https://panelharga.badanpangan.go.id/data/kabkota-range-by-levelharga/{}/3/{}"
        self.api_datetime_ : str  = self.get_api_date()
        self.prov_id_      : Dict = self.get_prov_id()
        self.kab_id_       : Dict = self.get_kab_id()

    def get_api_date(self) -> str:
        current_time = datetime.now()
        week_before = current_time - timedelta(days=7)

        current_date = current_time.strftime("%d-%m-%Y")
        week_before_date = week_before.strftime("%d-%m-%Y")

        api_date = week_before_date + "/" + current_date
        return api_date
    
    def get_prov_id(self) -> Dict:
        api_prov_id = self.api_prov_id_.format(self.api_datetime_)
        id_prov_json = requests.get(api_prov_id).json()

        prov_id = {}
        for prov in id_prov_json['data']:
            prov_id[prov['nama']] = prov['id']

        return prov_id
    
    def get_kab_id(self) -> Dict:
        kab_id = {}

        for prov_id in self.prov_id_.values():
            api_kab_id = self.api_kab_id_.format(prov_id, self.api_datetime_)
            id_kab_json = requests.get(api_kab_id).json()

            kab_iter = {}
            for kab in id_kab_json["data"]:
                kab_iter[kab["nama"]] = kab["id"]
                kab_id = kab_id | kab_iter
        
        return kab_id

class Province(PanelScraper):
    def __init__(self, province_name: str) -> None:
        super().__init__()
        self.prov_name_   : str = province_name
        self.prov_kab_id_ : Dict = self.get_prov_kab_id() 
        self.prov_prices  : Dict = self.get_prov_prices()
        self.available_ingredients : List = list(self.prov_prices.keys())

    def get_prov_kab_id(self) -> Dict:
        prov_kab_json = requests.get(self.api_kab_id_.format(self.prov_id_[self.prov_name_], self.api_datetime_)).json()

        prov_kab_id = {}
        for kab in prov_kab_json['data']:
            prov_kab_id[kab['nama']] = kab['id']

        return prov_kab_id
    
    def get_prov_prices(self) -> Dict:
        cost_prov_json = requests.get(self.api_prov_prices_.format(self.prov_id_[self.prov_name_], self.api_datetime_)).json()

        prices = {}
        for costs in cost_prov_json['data']:
            prices[costs['name']] = [cost['geomean'] for cost in costs['by_date']]

        return prices
    
    def get_prov_ingredient_price(self, ingredient_name: str, method: str = "latest"):
        ingredient_prices = self.prov_prices[ingredient_name]

        if method == "latest":
            for ingredient_price in ingredient_prices[::-1]:
                if ingredient_price != "-":
                    return ingredient_price
        else: 
            return ingredient_prices
    
class Kabupaten(PanelScraper):
    def __init__(self, kabupaten_name: str) -> None:
        super().__init__()
        self.kab_name_             : str  = "Kab. " + kabupaten_name
        self.kab_prices            : Dict = self.get_kab_prices()
        self.available_ingredients : List = list(self.kab_prices.keys())
    
    def get_kab_prices(self) -> Dict:
        cost_kab_json = requests.get(self.api_kab_prices_.format(self.kab_id_[self.kab_name_], self.api_datetime_)).json()
        
        prices = {}
        for costs in cost_kab_json['data']:
            prices[costs['name']] = [cost['geomean'] for cost in costs['by_date']]

        return prices
    
    def get_kab_ingredient_price(self, ingredient_name: str, method: str = "latest"):
        ingredient_prices = self.kab_prices[ingredient_name]

        if method == "latest":
            for ingredient_price in ingredient_prices[::-1]:
                if ingredient_price != "-":
                    return ingredient_price
        else: 
            return ingredient_prices

test api

Testing Province

In [102]:
jawa_tengah = Province(province_name="Jawa Tengah")

In [103]:
jawa_tengah.prov_prices

{'Beras Premium': [14840, 14830, 14830, 14770, 14840, 14880, 14860, 14830],
 'Beras Medium': [13060, 13090, 12990, 13050, 13030, 12940, 13080, 12980],
 'Kedelai Biji Kering (Impor)': [9690,
  9760,
  9630,
  9690,
  9620,
  9690,
  9650,
  9590],
 'Bawang Merah': [25460, 26240, 26540, 26740, 26670, 26970, 27130, 27750],
 'Bawang Putih Bonggol': [36160,
  36700,
  36700,
  36840,
  36790,
  36600,
  36900,
  36880],
 'Cabai Merah Keriting': [21430,
  21340,
  20990,
  21550,
  21330,
  21310,
  22070,
  21730],
 'Cabai Rawit Merah': [41050, 40810, 40450, 40660, 40650, 40250, 39770, 38440],
 'Daging Sapi Murni': [130440,
  130540,
  130190,
  130390,
  129890,
  129760,
  130660,
  130440],
 'Daging Ayam Ras': [35740, 35820, 36110, 36350, 36070, 36420, 36470, 36460],
 'Telur Ayam Ras': [25790, 25980, 25960, 25840, 25890, 26080, 25990, 26080],
 'Gula Konsumsi': [17150, 17110, 17080, 17140, 17120, 17110, 17110, 17150],
 'Minyak Goreng Kemasan Sederhana': [17210,
  17190,
  17230,
  17270,


In [104]:
jawa_tengah.get_prov_ingredient_price("Bawang Merah")

27750

Testing kab

In [95]:
banyumas = Kabupaten(kabupaten_name="Banyumas")
aceh_barat = Kabupaten(kabupaten_name="Aceh Barat")

In [96]:
aceh_barat.get_kab_ingredient_price("Bawang Merah", "gg")

[35000, 35990, 35000, 33470, 32500, 33990, 34500, 35000]

In [97]:
aceh_barat.get_kab_ingredient_price("Bawang Merah")

35000